In [1]:
%load_ext autoreload
%autoreload 2
import os
import datetime
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from pyspark.sql import functions as F
from pyspark.sql.types import *

from distribution_cost.configuration import spark_config
from distribution_cost.configuration.app import AppConfig
from distribution_cost.configuration.data import DataConfig
from distribution_cost.infra import oracle
from distribution_cost.domain import kpis

import numpy as np

/gpfs/user/e587246/dco00/conf/application.yml
/gpfs/user/e587246/dco00


In [2]:
# Database uri
app_config = AppConfig()
db_uri = app_config.db_uri_jdbc
db_uri

'jdbc:oracle:thin:BRC_E587247/uOR3hLZr@//pyox2k01:1521/BRCEX_PP2'

In [3]:
# Create spark session
spark_context, spark_session = spark_config.get_spark(app_name="app-distribution-cost",
                                                      executors=4, executor_cores=4, executor_mem='5g',
                                                      dynamic_allocation=True, max_executors=8)

# spark_session.conf.set("spark.sql.crossJoin.enabled", "true")

In [4]:
df_rbvqtlm1 = spark_session.read.load("/user/brc06/data/standardized/bds00/rbvqtlm1/")
df_rbvqttff = spark_session.read.load("/user/brc06/data/standardized/bds00/rbvqttff/")
df_rbvqtlm1.createOrReplaceTempView("df_rbvqtlm1")
df_rbvqttff.createOrReplaceTempView("df_rbvqttff")

In [5]:
queryMOTEUR = """
(SELECT m.cod_motor, m.energia, m.literal, f.libelle_francais
FROM df_rbvqtlm1 as m
JOIN df_rbvqttff f ON m.qi_filial = f.qi_filial
WHERE f.libelle_francais IN ('France')
)
"""

In [216]:
dfMOTEUR = spark_session.sql(queryMOTEUR)

In [217]:
column_value_Energy = {'ELECTRICO':'Electric','Diesel':'Diesel','DIESEL':'Diesel','ESSENCE':'Petrol','ELECTRIQUE':'Electric', 'Gasolina':'Petrol','GASOLINA': 'Petrol'}
dfMOTEUR = dfMOTEUR.replace(to_replace=column_value_Energy, subset=['energia'])
dfMOTEUR=dfMOTEUR.withColumn('energia', F.regexp_replace(F.col("energia"), "\\?", ""))
dfMOTEUR=dfMOTEUR.drop('literal')
dfMOTEUR=dfMOTEUR.drop_duplicates()
#dfMOTEUR=dfMOTEUR.drop_duplicates(['cod_motor','libelle_francais'])
dfMOTEUR=dfMOTEUR.withColumnRenamed('libelle_francais','Country')
duplicates = dfMOTEUR.groupby('cod_motor','Country').count()
dfMOTEUR= dfMOTEUR.join(duplicates, on=['cod_motor','Country'])
dfMOTEUR=dfMOTEUR.withColumnRenamed('count','count_duplicates')

In [218]:
dfMOTEUR = dfMOTEUR.withColumn('to_keep',F.when((F.col('energia')=='')&(F.col('count_duplicates')>1),0).otherwise(1))
dfMOTEUR=dfMOTEUR.filter(F.col('to_keep')==1).drop('to_keep','count_duplicates')

In [219]:
duplicates = dfMOTEUR.groupby('cod_motor','Country').count()
dfMOTEUR= dfMOTEUR.join(duplicates, on=['cod_motor','Country'])
dfMOTEUR=dfMOTEUR.withColumnRenamed('count','count_duplicates')
dfMOTEUR=dfMOTEUR.withColumn('energia',F.when(F.col('count_duplicates')>1,'Unknown').otherwise(F.col('energia')))
dfMOTEUR=dfMOTEUR.drop_duplicates().drop('count_duplicates')                           

In [220]:
# dfMOTEUR.filter(F.col('energia')=='').show()

+---------+-------+-------+
|cod_motor|Country|energia|
+---------+-------+-------+
|       RH| France|       |
|         | France|       |
|       QB| France|       |
|       QR| France|       |
+---------+-------+-------+



In [221]:
dfMOTEUR.write.jdbc(url=db_uri, table="SMKT005_REF_MOT", mode="overwrite")

In [222]:
dfMOTEUR.write.mode("overwrite").partitionBy("Country").parquet("hdfs:///user/e587247/data/refined/moteur/")